## Pruebas de Consolidación de OC

In [1]:
import pandas as pd
import pyodbc
import psycopg2 as pg2

# Cargar configuración DINAMICA de acuerdo al entorno
from dotenv import dotenv_values
import os
import sys
import time
import logging
import traceback

import io

ENV_PATH = os.environ.get("ETL_ENV_PATH", "E:/ETL/ETL_DIARCO/.env")  # Toma Producción si está definido, o la ruta por defecto E:\ETL\ETL_DIARCO\.env
# Verificar si el archivo .env existe
if not os.path.exists(ENV_PATH):
    print(f"El archivo .env no existe en la ruta: {ENV_PATH}")
    print(f"Directorio actual: {os.getcwd()}")
    sys.exit(1)
    
secrets = dotenv_values(ENV_PATH)
folder = f"E:/ETL/ETL_DIARCO/{secrets['FOLDER_DATOS']}"
folder_logs = f"E:/ETL/ETL_DIARCO/{secrets['FOLDER_LOG']}"
timestamp = pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")

In [2]:
# Funciones Locales
def Open_Connection():
    conn_str = f'DRIVER={secrets["SQLP_DRIVER"]};SERVER={secrets["SQLP_SERVER"]};PORT={secrets["SQLP_PORT"]};DATABASE={secrets["SQLP_DATABASE"]};UID={secrets["SQLP_USER"]};PWD={secrets["SQLP_PASSWORD"]}'
    # print (conn_str) 
    try:    
        conn = pyodbc.connect(conn_str)
        return conn
    except:
        print('Error en la Conexión')
        return None

def Open_Diarco_Data(): 
    conn_str = f"dbname={secrets['PG_DB']} user={secrets['PG_USER']} password={secrets['PG_PASSWORD']} host={secrets['PG_HOST']} port={secrets['PG_PORT']}"
    #print (conn_str)
    for i in range(5):
        try:    
            conn = pg2.connect(conn_str)
            return conn
        except Exception as e:
            print(f'Error en la conexión: {e}')
            time.sleep(5)
    return None  # Retorna None si todos los intentos fallan

def Close_Connection(conn): 
    if conn is not None:
        conn.close()
        # print("[OK] Conexión cerrada.")    
    return True

# -----------------------------------
print(f"PREPARANDO LOGS : Directorio actual: {os.getcwd()}")
print(f"[INFO] Cargando configuración desde: {ENV_PATH}")
print(f"[INFO] Carpeta de datos: {folder}") 
os.makedirs(folder_logs, exist_ok=True)

log_file = os.path.join(folder_logs, "publicacion_oc_precarga.log")

#
# Configurar logging
logging.basicConfig(
    filename=log_file,
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)


def limpiar_campos_oc(df: pd.DataFrame) -> pd.DataFrame:
    # --- Texto con longitudes destino ---
    df["c_usuario_genero_oc"]  = df["c_usuario_genero_oc"].fillna("").astype(str).str[:10]
    df["c_terminal_genero_oc"] = df["c_terminal_genero_oc"].fillna("").astype(str).str[:15]
    df["c_usuario_bloqueo"]    = df["c_usuario_bloqueo"].fillna("").astype(str).str[:10]
    df["m_procesado"]          = df["m_procesado"].fillna("N").astype(str).str[:1]
    df["c_compra_kikker"]      = df["c_compra_kikker"].fillna("").astype(str).str[:20]
    df["c_usuario_modif"]      = df["c_usuario_modif"].fillna("").astype(str).str[:20]

    # --- Claves y numéricos EXACTOS como INT ---
    for col in ["c_proveedor", "c_articulo", "c_sucu_empr", "u_prefijo_oc", "u_sufijo_oc", "c_comprador"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0).astype(int)

    # Cantidad a publicar en bultos/kilos como INT (>=0)
    if "q_bultos_kilos_diarco" in df.columns:
        df["q_bultos_kilos_diarco"] = pd.to_numeric(df["q_bultos_kilos_diarco"], errors="coerce").fillna(0)
        df["q_bultos_kilos_diarco"] = df["q_bultos_kilos_diarco"].clip(lower=0).astype(int)

    # --- Timestamps ---
    for dcol in ["f_alta_sist", "f_genero_oc", "f_procesado"]:
        if dcol in df.columns:
            df[dcol] = pd.to_datetime(df.get(dcol), errors='coerce') # type: ignore
    if "f_genero_oc" in df.columns:
        df["f_genero_oc"] = df["f_genero_oc"].fillna(pd.Timestamp('1900-01-01 00:00:00'))
    if "f_procesado" in df.columns:
        df["f_procesado"] = df["f_procesado"].fillna(pd.Timestamp('1900-01-01 00:00:00'))

    # --- Deduplicar intratable por PK destino ---
    pk_cols = ["c_proveedor", "c_articulo", "c_sucu_empr"]
    pk_cols = [c for c in pk_cols if c in df.columns]
    if pk_cols:
        df = df.drop_duplicates(subset=pk_cols, keep="last").reset_index(drop=True)

    return df

def forzar_enteros(df: pd.DataFrame) -> pd.DataFrame:
    # Columnas clave y numéricos exactos
    int_cols = [
        "c_proveedor", "c_articulo", "c_sucu_empr",
        "u_prefijo_oc", "u_sufijo_oc", "c_comprador",
        "q_bultos_kilos_diarco", "c_proveedor_primario",
        "abastecimiento"
    ]
    for col in int_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce").round().astype("Int64")

    # Booleanos
    if "m_publicado" in df.columns:
        df["m_publicado"] = df["m_publicado"].fillna(False).astype(bool)

    # Fechas (por si se degradaron)
    for dcol in ["f_alta_sist", "f_genero_oc", "f_procesado"]:
        if dcol in df.columns:
            df[dcol] = pd.to_datetime(df[dcol], errors="coerce")

    return df

def validar_longitudes(df):
    campos_texto = [
        "c_usuario_genero_oc", "c_terminal_genero_oc", "c_usuario_bloqueo",
        "m_procesado", "c_compra_kikker", "c_usuario_modif"
    ]
    print("\n [INFO] Validando longitudes máximas por columna de texto:")
    for col in campos_texto:
        max_len = df[col].astype(str).map(len).max()
        print(f"{col}: longitud máxima = {max_len}")

PREPARANDO LOGS : Directorio actual: e:\ETL\ETL_DIARCO\scripts\pull
[INFO] Cargando configuración desde: E:/ETL/ETL_DIARCO/.env
[INFO] Carpeta de datos: E:/ETL/ETL_DIARCO/data


In [3]:
 # 1. Conexión a PostgreSQL
conn_pg = Open_Diarco_Data()
if conn_pg is None:
    raise ConnectionError("[ERROR] No se pudo conectar a PostgreSQL")

query = """
SELECT *
FROM public.t080_oc_precarga_kikker
WHERE m_publicado = false
AND c_proveedor = 11740
"""
df_oc = pd.read_sql(query, conn_pg) # type: ignore



# if df_oc.empty:
#     logging.warning("[WARNING] No hay registros pendientes de publicación")
    #return

lista_proveedores = df_oc['c_proveedor'].dropna().astype(int).unique().tolist()

# Formatea la lista para usarla en la consulta SQL
in_clause = ', '.join([f"'{prov}'" for prov in lista_proveedores])


C:\Users\eduar\AppData\Local\Temp\ipykernel_33292\1451225749.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_oc = pd.read_sql(query, conn_pg) # type: ignore


In [4]:
# 1B. Traer productos vigentes de PostgreSQL
queryp = f"""
SELECT c_sucu_empr, c_articulo, c_proveedor_primario, abastecimiento, cod_cd
FROM src.base_productos_vigentes
WHERE c_proveedor_primario IN ({in_clause})
"""

df_prod = pd.read_sql(queryp, conn_pg) # type: ignore
# if df_prod.empty:
#     logging.warning("[WARNING] No hay productos relacionados")
#     return

df_prod = forzar_enteros(df_prod)

# Merge completo
df_completo = df_oc.merge(
    df_prod,
    how='left',
    left_on=['c_sucu_empr', 'c_articulo', 'c_proveedor'],
    right_on=['c_sucu_empr', 'c_articulo', 'c_proveedor_primario']
)
df_completo = forzar_enteros(df_completo)
if 'c_proveedor_primario' in df_completo.columns:
    df_completo.drop(columns=['c_proveedor_primario'], inplace=True)

C:\Users\eduar\AppData\Local\Temp\ipykernel_33292\4056776813.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_prod = pd.read_sql(queryp, conn_pg) # type: ignore


In [5]:
# -------------------------------
# PARTES A PUBLICAR
# -------------------------------
partes = []

# 41CD (consolidado a sucursal 41)
df_41 = df_completo[df_completo['cod_cd'] == '41CD']
if not df_41.empty:
    df_grouped_41 = df_41.groupby(['c_proveedor', 'c_articulo'], as_index=False).agg({
        'q_bultos_kilos_diarco': 'sum',
        'f_alta_sist': 'first',
        'c_usuario_genero_oc': 'first',
        'c_terminal_genero_oc': 'first',
        'f_genero_oc': 'first',
        'c_usuario_bloqueo': 'first',
        'm_procesado': 'first',
        'f_procesado': 'first',
        'u_prefijo_oc': 'first',
        'u_sufijo_oc': 'first',
        'c_compra_kikker': 'first',
        'c_usuario_modif': 'first',
        'c_comprador': 'first'
    }).reset_index(drop=True)
    df_grouped_41['c_sucu_empr'] = 41
    df_grouped_41['m_publicado'] = False
    partes.append(df_grouped_41)
    logging.info(f"[INFO] Registros consolidados para 41CD: {len(df_grouped_41)}")
else:
    logging.warning("[WARNING] No hay registros de cod_cd '41CD' para publicar")

In [6]:
# 82CD (consolidado a sucursal 82)
df_82 = df_completo[df_completo['cod_cd'] == '82CD']
if not df_82.empty:
    df_grouped_82 = df_82.groupby(['c_proveedor', 'c_articulo'], as_index=False).agg({
        'q_bultos_kilos_diarco': 'sum',
        'f_alta_sist': 'first',
        'c_usuario_genero_oc': 'first',
        'c_terminal_genero_oc': 'first',
        'f_genero_oc': 'first',
        'c_usuario_bloqueo': 'first',
        'm_procesado': 'first',
        'f_procesado': 'first',
        'u_prefijo_oc': 'first',
        'u_sufijo_oc': 'first',
        'c_compra_kikker': 'first',
        'c_usuario_modif': 'first',
        'c_comprador': 'first'
    }).reset_index(drop=True)
    df_grouped_82['c_sucu_empr'] = 82
    df_grouped_82['m_publicado'] = False
    partes.append(df_grouped_82)
    logging.info(f"[INFO] Registros consolidados para 82CD: {len(df_grouped_82)}")
else:
    logging.warning("[WARNING] No hay registros de cod_cd '82CD' para publicar")

In [7]:
# PASSTHROUGH (entregas directas: no consolidan)
# Tomamos todo lo que NO sea 41CD/82CD (incluye NULL en cod_cd) y lo dejamos tal cual.
mask_passthrough = (~df_completo['cod_cd'].isin(['41CD', '82CD'])) | (df_completo['cod_cd'].isna())
df_passthrough = df_completo.loc[mask_passthrough].copy()

In [8]:
if not df_passthrough.empty:
    # Homogeneizar columnas a las de las partes consolidadas
    cols_out = [
        'c_proveedor', 'c_articulo', 'c_sucu_empr', 'q_bultos_kilos_diarco',
        'f_alta_sist', 'c_usuario_genero_oc', 'c_terminal_genero_oc', 'f_genero_oc',
        'c_usuario_bloqueo', 'm_procesado', 'f_procesado', 'u_prefijo_oc',
        'u_sufijo_oc', 'c_compra_kikker', 'c_usuario_modif', 'c_comprador'
    ]
    # Asegurar existencia de columnas
    for c in cols_out:
        if c not in df_passthrough.columns:
            df_passthrough[c] = pd.NA

    df_passthrough = df_passthrough[cols_out].reset_index(drop=True)
    df_passthrough['m_publicado'] = False
    df_passthrough = forzar_enteros(df_passthrough)
    partes.append(df_passthrough)
    print(f"[INFO] Registros de entrega directa (passthrough): {len(df_passthrough)}")
else:
    print("[INFO] No hay registros de entrega directa (passthrough) para publicar")

# Si no hay nada que publicar, salir
if not partes:
    print("[WARNING] No hay registros con cod_cd {41CD,82CD} ni entregas directas para publicar")
    conn_pg.close()
    #return pd.DataFrame()


[INFO] Registros de entrega directa (passthrough): 45


In [ ]:
df_completo.columns

In [11]:
# -----------------------------------
# CONCATENACIÓN EXPLÍCITA (grouped + passthrough)
# -----------------------------------
frames = []

# Aseguramos variables y tamaños (0 si no existen o están vacías)
n_g41 = len(df_grouped_41) if 'df_grouped_41' in locals() and isinstance(df_grouped_41, pd.DataFrame) and not df_grouped_41.empty else 0
n_g82 = len(df_grouped_82) if 'df_grouped_82' in locals() and isinstance(df_grouped_82, pd.DataFrame) and not df_grouped_82.empty else 0
n_dir = len(df_passthrough) if 'df_passthrough' in locals() and isinstance(df_passthrough, pd.DataFrame) and not df_passthrough.empty else 0

if n_g41 > 0:
    frames.append(df_grouped_41)
if n_g82 > 0:
    frames.append(df_grouped_82)
if n_dir > 0:
    frames.append(df_passthrough)

if not frames:
    logging.warning("[WARNING] No hay partes para concatenar (41CD agrupado, 82CD agrupado ni directos)")
    conn_pg.close()
   # return pd.DataFrame()

# Concatenar uno debajo del otro en orden: 41 → 82 → directos
df_merged = pd.concat(frames, axis=0, ignore_index=True)

logging.info(
    f"[INFO] Total registros concatenados: {len(df_merged)} "
    f"(41CD_agrupados={n_g41}, 82CD_agrupados={n_g82}, Directos={n_dir})"
)


# Definir el orden deseado de columnas
orden_columnas = [
    'c_proveedor', 'c_articulo', 'c_sucu_empr', 'q_bultos_kilos_diarco',
    'f_alta_sist', 'c_usuario_genero_oc', 'c_terminal_genero_oc', 'f_genero_oc',
    'c_usuario_bloqueo', 'm_procesado', 'f_procesado',
    'u_prefijo_oc', 'u_sufijo_oc', 'c_compra_kikker', 'c_usuario_modif',
    'c_comprador'
]
# Reordenar el DataFrame
df_merged = df_merged[orden_columnas]


# Normalización final de tipos enteros/fechas/booleanos
df_merged = forzar_enteros(df_merged)

In [ ]:
df_merged.info()



In [ ]:
# Paso 1: Agrupar y sumar la cantidad
df_sumado = df_merged.groupby(['c_proveedor', 'c_articulo', 'c_sucu_empr'], as_index=False)['q_bultos_kilos_diarco'].sum()

# Paso 2: Eliminar duplicados del original (conservando la primera ocurrencia)
df_sin_duplicados = df_merged.drop_duplicates(subset=['c_proveedor', 'c_articulo', 'c_sucu_empr'], keep='first')

# Paso 3: Eliminar la columna original de cantidad para evitar conflicto
df_sin_duplicados = df_sin_duplicados.drop(columns=['q_bultos_kilos_diarco'])

# Paso 4: Reincorporar la cantidad sumada
df_final = df_sin_duplicados.merge(df_sumado, on=['c_proveedor', 'c_articulo', 'c_sucu_empr'], how='left')



## Hasta acá la generación de Datos

In [ ]:
 # 1C. Traer Stock CENTROS DE DISTRIBUCIÓN (solo afecta 41/82)
querystock = f"""
    SELECT 
        codigo_sucursal AS c_sucu_empr, 
        codigo_articulo AS c_articulo,
        P.q_factor_compra,
        COALESCE(stock, 0) AS stock_origen,
        COALESCE(pedido_pendiente, 0) AS pedido_pendiente, 
        COALESCE(transfer_pendiente, 0) AS transfer_pendiente, 
        FLOOR(
            (COALESCE(stock, 0) + COALESCE(pedido_pendiente, 0) + COALESCE(transfer_pendiente, 0)) 
            / COALESCE(P.q_factor_compra,1 )
        ) AS stock
    FROM src.base_stock_sucursal
    LEFT JOIN src.base_productos_vigentes P
        ON codigo_sucursal = c_sucu_empr 
        AND codigo_articulo  = c_articulo
    WHERE codigo_proveedor IN ({in_clause}) 
        AND codigo_sucursal IN (41, 82)
"""
df_stock = pd.read_sql(querystock, conn_pg)  # type: ignore
df_stock = forzar_enteros(df_stock)

In [ ]:
# Guardado de diagnósticos
try:
    df_completo.to_csv(os.path.join(folder_logs, f"{timestamp}_origen_completo.csv"),
                        index=False, encoding='utf-8-sig')
except Exception:
    pass

if not df_stock.empty:
    df_stock.rename(columns={'c_sucu_empr': 'c_sucu_empr_stock', 'c_articulo': 'c_articulo_stock'}, inplace=True)
    try:
        df_stock.to_csv(os.path.join(folder_logs, f"{timestamp}_stock.csv"),
                        index=False, encoding='utf-8-sig')
    except Exception:
        pass

    # Merge con stock por clave (sucursal, articulo)
    df_final = df_final.merge(
        df_stock[['c_sucu_empr_stock', 'c_articulo_stock', 'stock']],
        how='left',
        left_on=['c_sucu_empr', 'c_articulo'],
        right_on=['c_sucu_empr_stock', 'c_articulo_stock']
    )
    df_final.drop(columns=['c_sucu_empr_stock', 'c_articulo_stock'], inplace=True)

    # Ajuste de cantidad por stock (solo afectará a 41/82; directas no matchean y quedan sin descuento)
    if 'stock' in df_final.columns:
        df_final['q_bultos_kilos_diarco'] = (
            df_final['q_bultos_kilos_diarco'].fillna(0) - df_final['stock'].fillna(0)
        ).clip(lower=0).astype(int)


In [ ]:
# Limpieza de columnas si existen
for col in ['abastecimiento', 'cod_cd', 'stock']:
    if col in df_final.columns:
        df_final.drop(columns=[col], inplace=True)

# Filtrar cantidades > 0
if 'q_bultos_kilos_diarco' in df_final.columns:
    df_final = df_final[df_final['q_bultos_kilos_diarco'] > 0].reset_index(drop=True)

# Unicidad intra-lote por PK destino
pk_cols = ['c_proveedor', 'c_articulo', 'c_sucu_empr']
total = len(df_final)
unicas = len(df_final.drop_duplicates(subset=pk_cols))
if total != unicas:
    logging.warning(f"[WARN] Duplicados intra-batch detectados: {total - unicas}")
    try:
        df_final[df_final.duplicated(subset=pk_cols, keep=False)].to_csv(
            os.path.join(folder_logs, f"{timestamp}_duplicados_intra_batch.csv"),
            index=False, encoding='utf-8-sig'
        )
    except Exception:
        pass
    df_final = df_final.drop_duplicates(subset=pk_cols, keep='last').reset_index(drop=True)

# Guardar pedido consolidado
try:
    df_final.to_csv(os.path.join(folder_logs, f"{timestamp}_pedido_consolidado.csv"),
                        index=False, encoding='utf-8-sig')
except Exception:
    pass

In [ ]:
# Definir el orden deseado de columnas
orden_columnas = [
    'c_proveedor', 'c_articulo', 'c_sucu_empr', 'q_bultos_kilos_diarco',
    'f_alta_sist', 'c_usuario_bloqueo', 'm_procesado', 'f_procesado',
    'u_prefijo_oc', 'u_sufijo_oc', 'c_compra_kikker', 'c_usuario_modif',
    'c_comprador', 'm_publicado'
]
# Reordenar el DataFrame
df_final = df_final[orden_columnas]

In [ ]:
df_final.columns

In [ ]:
df_oc.columns

In [ ]:
df_merged.columns

In [ ]:
# Filtrar por cod_cd = '41CD'
df_directo = df_merged[df_merged['cod_cd'] != '41CD']


## PRUEBA BLOQUE

In [ ]:
def consolidar_oc_precarga():
    logging.info("[INFO] Iniciando consolidación por abastecimiento")  
    conn_pg = None

    try:
        # 1. Conexión a PostgreSQL
        conn_pg = Open_Diarco_Data()
        if conn_pg is None:
            raise ConnectionError("[ERROR] No se pudo conectar a PostgreSQL")
        
        query = """
        SELECT *
        FROM public.t080_oc_precarga_kikker
        WHERE m_publicado = false
        """
        df_oc = pd.read_sql(query, conn_pg) # type: ignore

        if df_oc.empty:
            logging.warning("[WARNING] No hay registros pendientes de publicación")
            return

        # Convertir a enteros antes de armar la cláusula IN
        lista_proveedores = (
            pd.to_numeric(df_oc['c_proveedor'], errors='coerce')
            .dropna()
            .astype(int)
            .unique()
            .tolist()
        )


        # Formatea la lista para usarla en la consulta SQL
        in_clause = ', '.join([f"'{prov}'" for prov in lista_proveedores])

        # 1B. Traer productos vigentes de PostgreSQL
        queryp = f"""
        SELECT c_sucu_empr, c_articulo, c_proveedor_primario, abastecimiento, cod_cd
        FROM src.base_productos_vigentes
        WHERE c_proveedor_primario IN ({in_clause})
        """
        df_prod = pd.read_sql(queryp, conn_pg) # type: ignore

        df_merged = df_oc.merge(
            df_prod,
            how='left',
            left_on=['c_sucu_empr', 'c_articulo', 'c_proveedor'],
            right_on=['c_sucu_empr', 'c_articulo', 'c_proveedor_primario']
        )

        # Filtrar por cod_cd = '41CD'
        df_41= df_merged[df_merged['cod_cd'] == '41CD']
        if df_41.empty:
            logging.warning("[WARNING] No hay registros de cod_cd '41CD' para publicar")
        else:
            df_grouped_41 = df_41.groupby(
            ['c_proveedor', 'c_articulo'],
            as_index=False
                ).agg({
                    'q_bultos_kilos_diarco': 'sum',
                    'f_alta_sist': 'first',
                    'c_usuario_genero_oc': 'first',
                    'c_terminal_genero_oc': 'first',    
                    'f_genero_oc': 'first',
                    'c_usuario_bloqueo': 'first',
                    'm_procesado': 'first',
                    'f_procesado': 'first',
                    'u_prefijo_oc': 'first',
                    'u_sufijo_oc': 'first',
                    'c_compra_kikker': 'first',
                    'c_usuario_modif': 'first',
                    'c_comprador': 'first'
                }).reset_index(drop=True)

            df_grouped_41['c_sucu_empr'] = 41
            # Borrar en Origen
            df_merged.drop(df_merged[df_merged['cod_cd'] == '41CD'].index, inplace=True)
            # Publicar en Destino
            df_merged = pd.concat([df_merged, df_grouped_41], ignore_index=True)

        # Filtrar por cod_cd = '82CD'
        df_82= df_merged[df_merged['cod_cd'] == '82CD']
        if df_82.empty:
            logging.warning("[WARNING] No hay registros de cod_cd '82CD' para publicar")
        else:
            df_grouped_82 = df_82.groupby(
            ['c_proveedor', 'c_articulo'],
            as_index=False
                ).agg({
                    'q_bultos_kilos_diarco': 'sum',
                    'f_alta_sist': 'first',
                    'c_usuario_genero_oc': 'first',
                    'c_terminal_genero_oc': 'first',    
                    'f_genero_oc': 'first',
                    'c_usuario_bloqueo': 'first',
                    'm_procesado': 'first',
                    'f_procesado': 'first',
                    'u_prefijo_oc': 'first',
                    'u_sufijo_oc': 'first',
                    'c_compra_kikker': 'first',
                    'c_usuario_modif': 'first',
                    'c_comprador': 'first'
                }).reset_index(drop=True)

            df_grouped_82['c_sucu_empr'] = 82
            # Borrar en Origen
            df_merged.drop(df_merged[df_merged['cod_cd'] == '82CD'].index, inplace=True)
            # Publicar en Destino
            df_merged = pd.concat([df_merged, df_grouped_82], ignore_index=True)

        
        # 1C. Traer Stock CENTROS DE DISTRIBUCIÓN EN BULTOS
        querystock = f"""
         SELECT S.c_sucu_empr ,S.c_articulo ,S.q_peso_articulo ,P.q_factor_proveedor
	            ,S.q_unid_articulo / p.q_factor_proveedor as stock
            FROM src.t060_stock S
            LEFT JOIN src.t052_articulos_proveedor P
                ON S.c_articulo = P.c_articulo
                WHERE P.c_proveedor in ({in_clause}) 
                and S.c_sucu_empr IN(41, 82)
        """

        df_stock = pd.read_sql(querystock, conn_pg) # type: ignore
        if df_stock.empty:
            logging.warning("[WARNING] No hay stock disponible para los proveedores seleccionados")
        else:
            df_stock.rename(columns={'c_sucu_empr': 'c_sucu_empr_stock', 'c_articulo': 'c_articulo_stock'}, inplace=True)
            # Hacemos el merge con clave múltiple
            # Esto agrega el stock a df_merged
            df_merged = df_merged.merge(
                df_stock[['c_sucu_empr_stock', 'c_articulo_stock', 'stock']],
                how='left',
                left_on=['c_sucu_empr', 'c_articulo'],
                right_on=['c_sucu_empr_stock', 'c_articulo_stock']
            )
            # Restar a q_bultos_kilos_diarco stock y tranformar a entero
            df_merged['q_bultos_kilos_diarco'] = (
                df_merged['q_bultos_kilos_diarco'] - df_merged['stock']
                ).clip(lower=0).astype(int) 
            # Eliminar columnas de stock
            df_merged.drop(columns=['c_sucu_empr_stock', 'c_articulo_stock', 'stock'], inplace=True)

        conn_pg.close()
        return df_merged 

    except Exception as e:
        logging.error("[ERROR] Error durante la CONSOLIDACIÓN de OC Precarga")
        logging.error(traceback.format_exc())
        print("[ERROR] Error durante la ejecución:", e)


In [ ]:
def publicar_oc_precarga():
    logging.info("[INFO] Iniciando publicación de OC Precarga")
    df_oc = consolidar_oc_precarga()
    if df_oc is None or df_oc.empty:
        logging.warning("[WARNING] No hay registros consolidados para publicar")
        return

    # Abrir conexión a PostgreSQL SOLO para el update
    conn_pg = Open_Diarco_Data()
    if conn_pg is None:
        raise ConnectionError("[ERROR] No se pudo reconectar a PostgreSQL para actualizar publicados")

    conn_sql = None
    cursor_sql = None

    try:
        
        if df_oc.empty:
            logging.warning("[WARNING] No hay registros pendientes de publicación")
            return

        # Agrupar por c_proveedor_primario y c_articulo, sumando las cantidades
        total_rows = len(df_oc)
        logging.info(f"[INFO] Registros a publicar: {total_rows}")

        df_oc = limpiar_campos_oc(df_oc)
        validar_longitudes(df_oc)
        print(df_oc.head(5))

        # 2. Conexión a SQL Server
        conn_sql = Open_Connection()
        if conn_sql is None:
            raise ConnectionError("[ERROR] No se pudo conectar a SQL Server")

        cursor_sql = conn_sql.cursor()
        cursor_sql.fast_executemany = True  # validar si es soportado por tu driver

        insert_stmt = """
        INSERT INTO [dbo].[T080_OC_PRECARGA_KIKKER] (
            [C_PROVEEDOR], [C_ARTICULO], [C_SUCU_EMPR], [Q_BULTOS_KILOS_DIARCO],
            [F_ALTA_SIST], [C_USUARIO_GENERO_OC], [C_TERMINAL_GENERO_OC], [F_GENERO_OC],
            [C_USUARIO_BLOQUEO], [M_PROCESADO], [F_PROCESADO], [U_PREFIJO_OC],
            [U_SUFIJO_OC], [C_COMPRA_KIKKER], [C_USUARIO_MODIF], [C_COMPRADOR]
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """

        data_tuples = df_oc[[
            'c_proveedor', 'c_articulo', 'c_sucu_empr', 'q_bultos_kilos_diarco',
            'f_alta_sist', 'c_usuario_genero_oc', 'c_terminal_genero_oc', 'f_genero_oc',
            'c_usuario_bloqueo', 'm_procesado', 'f_procesado', 'u_prefijo_oc',
            'u_sufijo_oc', 'c_compra_kikker', 'c_usuario_modif', 'c_comprador'
        ]].itertuples(index=False, name=None)

        cursor_sql.executemany(insert_stmt, list(data_tuples))
        conn_sql.commit()

        logging.info("[INFO] Inserción completada en SQL Server")
        print(f"✔ Se insertaron {total_rows} registros en SQL Server.")

        # 3. Marcar como publicados en PostgreSQL
        lista_compra_kikker = df_oc['c_compra_kikker'].dropna().unique().tolist()
        placeholders = ', '.join(['%s'] * len(lista_compra_kikker))
        update_stmt = f"""
                UPDATE public.t080_oc_precarga_kikker
                SET m_publicado = true
                WHERE c_compra_kikker IN ({placeholders})
            """

        with conn_pg.cursor() as cursor_pg:
            cursor_pg.execute(update_stmt, lista_compra_kikker)
            rows_updated = cursor_pg.rowcount
            conn_pg.commit()

        logging.info(f"[INFO] {rows_updated} registros marcados como publicados")
        print(f"✔ {rows_updated} registros actualizados con m_publicado = true")

    except Exception as e:
        logging.error("[ERROR] Error durante la publicación de OC Precarga")
        logging.error(traceback.format_exc())
        print("[ERROR] Error durante la ejecución:", e)

    finally:
        if cursor_sql:
            try:
                cursor_sql.close()
            except Exception as e:
                logging.warning(f"[WARNING] Error al cerrar cursor SQL: {e}")
        if conn_sql:
            try:
                conn_sql.close()
            except Exception as e:
                logging.warning(f"[WARNING] Error al cerrar conexión SQL Server: {e}")
        if conn_pg:
            try:
                conn_pg.close()
            except Exception as e:
                logging.warning(f"[WARNING] Error al cerrar conexión PostgreSQL: {e}")

In [ ]:
publicar_oc_precarga()
print(f"[INFO] Proceso finalizado. Ver log en: {log_file}")